In [ ]:
!pip install sqlmodel

In [ ]:
from typing import Optional

from sqlmodel import (
    Field,
    SQLModel,
    text,
    Relationship,
    create_engine
)
import uuid


class Hero(SQLModel, table=True):
    id: Optional[uuid.UUID] = Field(
        default=None,
        primary_key=True,
        index=True,
        nullable=False,
        sa_column_kwargs={"server_default": text("gen_random_uuid()"), "unique": True},
    )
    team_id: Optional[uuid.UUID] = Field(
        default=None,
        index=True,
        foreign_key="team.id",
    )
    name: str
    secret_name: str
    age: Optional[int] = None

In [ ]:
import sqlite3

In [ ]:

c = sqlite3.connect('test.db')

url = "sqlite:////content/test.db"

engine = create_engine(url, echo=True)


def create_db_and_tables():
    SQLModel.metadata.create_all(engine)


if __name__ == "__main__":
    create_db_and_tables()
